In [ ]:
import pandas as pd
import numpy as np
from backtesting import arima_garch_python as models
from backtesting.generate_schedules import generate_schedule
import matplotlib.pyplot as plt
from common.config import schedules_generation_dates

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None


In [ ]:
x = models.ArimaGarchAnalyzer('default', 10000, [pd.to_datetime('1996-01-02')], [pd.to_datetime('2023-12-29')], log_to_file=True, log_level='info')
x.read_output_from_R()


In [ ]:
n_regimes = [2]  # 2, 3 - liczna dozwolonych reżimów zmienności
oos = ['2M']  # 1M, 2M, 3M, 6M, 12M - długość okresu out-of-sample
moving = True  # parametr wskazujący czy okres in-sample ma się rozszerzać, czy przesuwać zachowując długość 10 lat


In [ ]:
names = []
final_object = models.ArimaGarchAnalyzer(cost='default', start_capital=10000, start_dte=[pd.to_datetime('2006-01-03')],
                                         end_dte=[pd.to_datetime('2023-12-29')], log_to_file=True)
for interval in oos:
    schedules = schedules_generation_dates.get(interval)

    for n in n_regimes:
        x.optimise_strategy(schedules=schedules, interval=interval, moving=moving, n_regimes=n)
        df_temp = x.df_final
        del x.df_final
        _name = 'df_final_' + str(interval) + '_' + str(n) + '_' + str(2013 if moving else 1996)
        df_temp.to_csv(_name + '.csv')
        setattr(final_object, _name, df_temp)
        
        names.append(_name)
        